## Кластеризация

Кластеризация - один из видов алгоритмов машинного обучения без учителя. В этом случае в обучающей выборке отсутствуют значения целевых переменных. Имеются только признаки обучающих объектов. Задача кластеризации - сгруппировать похожие объекты в сгустки, которые называются кластерами. Кластеризация может быть использована, например, в качестве предварительного этапа для определения структуры данных. Одно из практических применений кластеризации - это сжатие изображений. С помощью нее можно уменьшить количество цветов. Для этого необходимо раскластеризовать цвета пикселей изображения, а затем заменить оригинальные цвета пискселей на центроиды получившихся кластеров, соответвующих пикселям.

В настоящей лабораторной работе Вам необходимо реализовать алгоритм кластеризации k-средних (k-means) и применить его для сжатия изображения.

In [49]:
# Здесь происходит закгрузка изображения
# Вы можете скачать любое изображение из интернета или же загрузить стандартные тестовые изображения из skimage.data
# Алгоритм k-means имеет достаточно большую сложность, поэтому рекомендуется использовать небольшие изображения для отладки,
# не более 128x128 пикселей. 

import skimage
import skimage.data
from skimage.io import imread

#img_path = "03.jpg"
#image = imread(img_path)
image = skimage.data.astronaut()

# Сохраняет исходное изображения под другим имененм. Просто для отладки.
save_path = "orig_image.jpg"
skimage.io.imsave(save_path, image)

# Создаем из исходного изображения матрицу ширина х высота х 3 (количество цеветов формата rgb)
img_float = skimage.img_as_float(image)
img_float.shape

(512L, 512L, 3L)

In [50]:
img_float

array([[[ 0.60392157,  0.57647059,  0.59215686],
        [ 0.42745098,  0.40392157,  0.48627451],
        [ 0.24705882,  0.22745098,  0.4       ],
        ..., 
        [ 0.49803922,  0.47058824,  0.45098039],
        [ 0.47058824,  0.45882353,  0.41568627],
        [ 0.49019608,  0.46666667,  0.43137255]],

       [[ 0.69411765,  0.67058824,  0.67058824],
        [ 0.56470588,  0.55294118,  0.56078431],
        [ 0.44313725,  0.44705882,  0.48627451],
        ..., 
        [ 0.49803922,  0.4627451 ,  0.43921569],
        [ 0.48627451,  0.45098039,  0.42352941],
        [ 0.4745098 ,  0.45490196,  0.41176471]],

       [[ 0.78823529,  0.76078431,  0.75686275],
        [ 0.71372549,  0.69803922,  0.68627451],
        [ 0.65882353,  0.64705882,  0.64313725],
        ..., 
        [ 0.50196078,  0.47058824,  0.45882353],
        [ 0.49411765,  0.45490196,  0.43921569],
        [ 0.48627451,  0.44705882,  0.42745098]],

       ..., 
       [[ 0.72941176,  0.66666667,  0.69019608],
        

In [51]:
# Разворачиваем исходную матрицу в список обучающих примеров: кол-во пикселей х 3. Получается, что цвета
# предсталяют собой признаки каждого обучающего объекта.

X = img_float.reshape(img_float.shape[0] * img_float.shape[1], 3)
X[:10]

array([[ 0.60392157,  0.57647059,  0.59215686],
       [ 0.42745098,  0.40392157,  0.48627451],
       [ 0.24705882,  0.22745098,  0.4       ],
       [ 0.21176471,  0.2       ,  0.38431373],
       [ 0.29803922,  0.29803922,  0.41568627],
       [ 0.39215686,  0.39215686,  0.40784314],
       [ 0.48627451,  0.4745098 ,  0.47843137],
       [ 0.54509804,  0.52941176,  0.52156863],
       [ 0.58039216,  0.55294118,  0.54117647],
       [ 0.55294118,  0.5254902 ,  0.50980392]])

## Заготовка класса для k-means

Используйте нижележащую заготовку для реализации алгоритма k-means. Вам необходимо реализовать функции _compute_centroids и _determine_closest_centroids. Также рекомендуется реализовать функцию _quality для отладки алгоритма. Формула расчета центроидов кластеров: 

$ \mu_y = \frac{\sum_{i=1}^l[y^i=y]x^i}{\sum_{i=1}^l[y^i=y]} $ - усредненное значение векторов, отнесенных к кластеру y.

Функциоанл качества кластеризации:

$ Q = \frac{1}{l}\sum_{i=1}^l(\rho(x^i,\mu_{y^i}))^2 $, где $\rho$ - евклидово расстояние.

In [52]:
import numpy as np
l1 = np.array([1,1,2,3,3,4,5,6,4])
l2 = np.array([1,1,2,3,3,4,5,6,4])
sum([l1,l2])

array([ 2,  2,  4,  6,  6,  8, 10, 12,  8])

In [53]:
import logging
import random

import numpy as np
import skimage.data

class SimpleKMeans:
    def __init__(self, k, max_iterations=100, tol=0.0001, verbose=False, qual=False):
        self.k_ = k
        self.max_iterations_ = max_iterations
        self.centroids_ = np.array([])
        self.tol_ = tol
        self.verobose_ = verbose
        self.qual_ = qual
        if verbose:
            self.logger = logging.getLogger(self.__class__.__name__)
            self.logger.setLevel(logging.DEBUG)

    # Функция принимает на вход обучающие объекты X, массив с целочисленными идентификаторами кластеров idx,
    # количество кластеров k. Функция возвращает центроиды класторов.
    def _compute_centroids(self, X, idx, k):
        centroids = np.zeros((k, 3))
        for i in range(k):
            aver = sum([X[j] for j in range(X.shape[0]) if idx[j] == i])
            num = np.count_nonzero(idx == i)
            if num > 0:
                centroids[i] = aver / num
        return centroids

    def _init_centroids(self, X, k):
        indx = random.sample(range(X.shape[0]), k)
        return X[indx]

    # Функция принимает на вход обучающие объекты X, центроиды кластеров centroids, количество кластеров k.
    # Функция вовращает список целочисленных идентификаторов кластеров для каждого объекта из X (len(X) == len(idx)).
    # При назначении кластера объекту ищется тот кластер, квадрат расстояния от объекта до центроида которого
    # - наименьшее. Используйте евклидово расстояние.
    def _determine_closest_centroids(self, X, centroids, k):
        idx = np.zeros(X.shape[0], dtype=np.int)
        for i in range(X.shape[0]):
            ros = [np.linalg.norm(centroids[j] - X[i]) for j in range(k)]
            idx[i] = np.argmin(ros)
        return idx

    # Функция возращает значения функционала качества кластеризации для текующих центроидов кластеров.
    def _quality(self, X, centroids, idx):
        Q = sum([np.linalg.norm(centroids[idx[j]] - X[j]) for j in range(X.shape[0])]) / X.shape[0]
        return Q

    def fit(self, X):
        centroids = self._init_centroids(X, self.k_)
        self.centroids_ = np.copy(centroids)
        for n in range(self.max_iterations_):
            if self.verobose_:
                self.logger.debug("Iteration: %d", n)

            idx = self._determine_closest_centroids(X, centroids, self.k_)
            centroids = self._compute_centroids(X, idx, self.k_)

            if self.qual_:
                qual = self._quality(X, centroids, idx)
                self.logger.debug("Qual: %f", qual)

            diff = np.linalg.norm(self.centroids_ - centroids)
            if self.verobose_:
                self.logger.debug("Diff: %f", diff)

            self.centroids_ = np.copy(centroids)

            if diff < self.tol_:
                break

    def predict(self, X):
        return self._determine_closest_centroids(X, self.centroids_, self.k_)

In [54]:
# Обучаем кластеризатор, чтобы он мог преобразовать все цвета изображения в k=16 цветов. 
# Когда verbose = True в ходе бучения будут распечатываться логи обучения.
# При qual = True будут также распечатываться значения фукнционала качества. Обратите внимание, что 
# значения функционала качества должны уменьшаться на каждой итерации, в противном случае в Вашем 
# коде присутсвует ошибка.

cl = SimpleKMeans(k = 16, max_iterations=100, tol=0.01, verbose = False)
cl.fit(X)

In [55]:
# Определяем идентификаторы каждого пискселя (номер цвета).

res = cl.predict(X)
res[50:60]

array([11, 11, 11, 11, 11, 11, 11, 13, 13, 11])

In [56]:
# Преобразуем цвета пикселей изображения. Восстанавливаем исходную матрицу изображения ширина х высота х 3.
# Сохарняем получившееся изображение.

X_transformed =  cl.centroids_[res]
new_image = X_transformed.reshape(img_float.shape[0], img_float.shape[1], 3)
new_save_path = "transformed_image.jpg"
skimage.io.imsave(new_save_path, new_image)